<a href="https://colab.research.google.com/github/vanessajen/FlatLay-Generator/blob/main/FlatLay%20Generator%20V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# FlatLay Generator V1 - Full Code

# 1. Install necessary libraries
!pip install diffusers==0.20.2 controlnet_aux==0.0.9 transformers==4.26.1 accelerate==0.17.0 huggingface_hub==0.13.3 --quiet

# 2. Import libraries
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from controlnet_aux import CannyDetector
from PIL import Image
import numpy as np
import cv2
import os
from google.colab import drive

# 3. Mount Google Drive
drive.mount('/content/drive')

# 4. Set Input/Output Paths
INPUT_FOLDER = "/content/drive/MyDrive/FlatLay_Input"
OUTPUT_FOLDER = "/content/drive/MyDrive/FlatLay_Demo_Output"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# 5. Load ControlNet and Stable Diffusion pipeline
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny",
    torch_dtype=torch.float16
)
pipeline = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16
)
pipeline.scheduler = UniPCMultistepScheduler.from_config(pipeline.scheduler.config)
pipeline.enable_xformers_memory_efficient_attention()
pipeline.to('cuda')

# 6. Initialize Canny Detector
canny = CannyDetector()

# 7. Define accessories list
accessories = [
    "Strawberry iced tea",
    "Pastel press-on nails",
    "AirPods",
    "Minimalism jewelry",
    "Baseball cap",
    "Sunglasses",
    "Mini Polaroid photo",
    "Fujifilm Instax camera",
    "Chanel perfume",
    "Sheer scarf",
    "Glow mist",
    "Blush compact",
    "Film camera",
    "Magazine",
    "Pastel flower hair clip",
    "Gold rings",
    "Dainty necklace",
    "Raffia tote bag"
]

# 8. Function to generate FlatLay image
def generate_flatlay(input_path, mode="Fast"):
    image = Image.open(input_path).convert("RGB")
    image = image.resize((512, 512))

    canny_image = canny(image)

    base_prompt = "a minimalistic flat lay aesthetic on white background, fashion product centered, surrounded by accessories: "
    accessory_prompt = ", ".join(np.random.choice(accessories, 3, replace=False))

    prompt = base_prompt + accessory_prompt + ", pastel tones, clean layout, high quality photo"

    if mode == "Fast":
        num_inference_steps = 20
        guidance_scale = 7.5
    else:  # Pro mode
        num_inference_steps = 30
        guidance_scale = 9.0

    output = pipeline(
        prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        image=canny_image
    ).images[0]

    return output

# 9. Process all images in Input Folder
input_images = [f for f in os.listdir(INPUT_FOLDER) if f.endswith(('.png', '.jpg', '.jpeg'))]

print(f"Found {len(input_images)} image(s) in FlatLay_Input folder:")

for img_name in input_images:
    input_path = os.path.join(INPUT_FOLDER, img_name)
    output_path = os.path.join(OUTPUT_FOLDER, f"flatlay_{img_name}")

    flatlay_img = generate_flatlay(input_path, mode="Fast")  # Default is Fast Mode
    flatlay_img.save(output_path)

print("\ud83c\udf80 Done processing all images!")